In [2]:
from lib.main_func_p1 import path
import pandas as pd

In [50]:
uniprot_data = ['P49841', 'kinase']
# uniprot_data = ['P22303', 'Non-IDG Family Targets']
uniprot_id = uniprot_data[0]
path_file = path(uniprot_id)

In [99]:
fingerprint = 'maccs'
n_splits = 10
resample_factor = 1.75
name_grid_file = f'{path_file}_RFgrid_{fingerprint}_s{n_splits}_r{resample_factor}.csv'

df = pd.read_csv(name_grid_file)
df.head()

,n_estimators,min_samples_split,min_samples_leaf,max_features,max_leaf_nodes,oob_score,max_samples,criterion,auc_score_train,accuracy_score_train,...,prec_score_train,confusion_m_train,auc_score_test,accuracy_score_test,sens_score_test,spec_score_test,prec_score_test,f1_score_test,confusion_m_test,time
0,100,4,20,auto,160,False,0.7,entropy,0.892216,0.790383,...,0.872727,"[[812, 35], [244, 240]]",0.852312,0.762763,0.504132,0.910377,0.762500,0.606965,"[[193, 19], [60, 61]]",4.77
1,100,8,10,auto,120,False,NaN,entropy,0.944381,0.854996,...,0.892183,"[[807, 40], [153, 331]]",0.904959,0.837838,0.685950,0.924528,0.838384,0.754545,"[[196, 16], [38, 83]]",5.30
2,80,8,15,0.6,180,False,0.7,entropy,0.929892,0.850488,...,0.832168,"[[775, 72], [127, 357]]",0.894355,0.819820,0.735537,0.867925,0.760684,0.747899,"[[184, 28], [32, 89]]",6.69
3,60,8,20,0.8,60,True,NaN,entropy,0.921683,0.842224,...,0.812785,"[[765, 82], [128, 356]]",0.885954,0.801802,0.727273,0.844340,0.727273,0.727273,"[[179, 33], [33, 88]]",6.90
4,60,2,25,1.0,180,False,NaN,entropy,0.909414,0.824944,...,0.780761,"[[749, 98], [135, 349]]",0.877612,0.798799,0.727273,0.839623,0.721311,0.724280,"[[178, 34], [33, 88]]",7.45


In [102]:
# Filtrar auc_score_train < limit_sup
df_score = df.copy()
df_score = df.drop(columns=['n_estimators', 'min_samples_split', 'min_samples_leaf', 'max_features', 'max_leaf_nodes', 'oob_score', 'max_samples',
                            'criterion','accuracy_score_train', 'sens_score_train',  'spec_score_train',  'prec_score_train',  'confusion_m_train',  'time'])
df_score['ratio'] = round(df_score.auc_score_test / df_score.auc_score_train, 2)
df_score = df_score.sort_values(by=['ratio', 'auc_score_train'], ascending=False)
df_score = df_score[df_score['auc_score_train'] <= 0.96]
df_score = df_score[df_score['sens_score_test'] >= 0.75]
print(df_score[['auc_score_train', 'auc_score_test']].head(10))
top10 = list(df_score[df_score['sens_score_test'] >= 0.75].iloc[:10].index)
df.iloc[top10]

        auc_score_train  auc_score_test
108060         0.941028        0.908799
20362          0.939953        0.908877
86902          0.939323        0.908292
51514          0.939031        0.907551
19863          0.939013        0.908019
61118          0.938493        0.905758
95745          0.938384        0.906849
98673          0.938014        0.905797
3335           0.937915        0.905387
39470          0.937893        0.906167


,n_estimators,min_samples_split,min_samples_leaf,max_features,max_leaf_nodes,oob_score,max_samples,criterion,auc_score_train,accuracy_score_train,...,prec_score_train,confusion_m_train,auc_score_test,accuracy_score_test,sens_score_test,spec_score_test,prec_score_test,f1_score_test,confusion_m_test,time
108060,80,8,15,0.6,160,True,NaN,entropy,0.941028,0.857250,...,0.838710,"[[777, 70], [120, 364]]",0.908799,0.831832,0.768595,0.867925,0.768595,0.768595,"[[184, 28], [28, 93]]",8.89
20362,60,8,15,0.7,140,True,NaN,entropy,0.939953,0.865515,...,0.841163,"[[776, 71], [108, 376]]",0.908877,0.825826,0.752066,0.867925,0.764706,0.758333,"[[184, 28], [30, 91]]",7.36
86902,80,4,15,0.7,100,False,NaN,entropy,0.939323,0.858753,...,0.833333,"[[773, 74], [114, 370]]",0.908292,0.825826,0.752066,0.867925,0.764706,0.758333,"[[184, 28], [30, 91]]",8.73
51514,60,8,15,0.7,160,False,NaN,gini,0.939031,0.863261,...,0.843182,"[[778, 69], [113, 371]]",0.907551,0.828829,0.760331,0.867925,0.766667,0.763485,"[[184, 28], [29, 92]]",6.74
19863,80,8,15,0.6,200,True,NaN,gini,0.939013,0.864012,...,0.849885,"[[782, 65], [116, 368]]",0.908019,0.840841,0.768595,0.882075,0.788136,0.778243,"[[187, 25], [28, 93]]",8.56
61118,100,6,15,0.7,160,True,0.9,entropy,0.938493,0.864763,...,0.850230,"[[782, 65], [115, 369]]",0.905758,0.825826,0.752066,0.867925,0.764706,0.758333,"[[184, 28], [30, 91]]",11.10
95745,100,4,15,0.7,180,False,NaN,gini,0.938384,0.863261,...,0.846330,"[[780, 67], [115, 369]]",0.906849,0.834835,0.760331,0.877358,0.779661,0.769874,"[[186, 26], [29, 92]]",10.53
98673,60,6,15,0.7,140,True,0.9,entropy,0.938014,0.862509,...,0.841270,"[[777, 70], [113, 371]]",0.905797,0.828829,0.752066,0.872642,0.771186,0.761506,"[[185, 27], [30, 91]]",7.05
3335,140,8,15,0.7,60,False,NaN,gini,0.937915,0.860255,...,0.843318,"[[779, 68], [118, 366]]",0.905387,0.834835,0.760331,0.877358,0.779661,0.769874,"[[186, 26], [29, 92]]",14.22
39470,120,6,15,0.7,40,True,NaN,gini,0.937893,0.861007,...,0.839002,"[[776, 71], [114, 370]]",0.906167,0.825826,0.760331,0.863208,0.760331,0.760331,"[[183, 29], [29, 92]]",13.52


In [103]:
lista_parametros = dict(df.iloc[top10[0]][['n_estimators', 'min_samples_split', 'min_samples_leaf', 'max_features', 'max_leaf_nodes', 'oob_score', 'max_samples', 'criterion']])
try:
    lista_parametros['max_features'] = float(lista_parametros['max_features'])
except:
    pass
print(lista_parametros)

{'n_estimators': 80, 'min_samples_split': 8, 'min_samples_leaf': 15, 'max_features': 0.6, 'max_leaf_nodes': 160, 'oob_score': True, 'max_samples': nan, 'criterion': 'entropy'}
